# custom loss function in tensorﬂow/keras to train a model for binary classiﬁcation and apply l2 reguralisation on only convolution and dense layers. Only the custom loss function is required as solution.

Loss functions, also called error functions, assess how close a model's predictions are to the actual target values. A lower loss indicates better model performance by showing that the model's predictions are closer to the desired target values. 
Below are some Loss function use for the Binary Class Classification

### 1. Squared hinge loss: 
Squared hinge loss is a function tailored for "maximum margin" binary classification tasks. It smoothens the error function's surface, simplifying numerical computations. For proper use, the target variable should be adjusted to belong to the set of {-1, 1}. 
It is recommended to utilize this loss function alongside the tanh() activation function in the final layer.

In [ ]:
import tensorflow as tf

def hinge_loss(y_true, y_pred):
    # Modify target labels to be in the set of {-1, 1}
    y_true = 2 * y_true - 1
    
    # Calculate Hinge Loss
    loss = tf.maximum(0., 1. - y_true * y_pred)
    
    return loss, tf.reduce_mean(loss)


### 2. Cross-Entropy/Logistic Loss (CE)
Cross-entropy loss, also called logistic loss, is frequently used in binary classification tasks with classes 0 and 1. It measures the difference between the actual class and the predicted value, usually ranging from 0 to 1. For best results.
it's recommended to use the sigmoid activation function as the final layer before applying cross-entropy loss.

In [ ]:
'''sigmoid_cross_entropy_loss'''
def sigmoid_cross_entropy_loss(y_true, y_pred):
    # Apply sigmoid function to predicted values
    y_pred = tf.math.sigmoid(y_pred)
    
    # Clip values to avoid extremes of the log function
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
    
    # Compute the binary cross-entropy loss
    term_0 = y_true * tf.math.log(y_pred)
    term_1 = (1 - y_true) * tf.math.log(1 - y_pred)
    loss = -(term_0 + term_1)
    return tf.reduce_mean(loss)

In [ ]:
'''weighted_binary_crossentropy'''
def weighted_binary_crossentropy(y_true, y_pred, pos_weight):
    epsilon = 1e-7  # Small constant to avoid log(0) or log(1)
    
    # Clip predicted values to avoid log(0) or log(1)
    y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
    
    # Calculate binary cross-entropy loss with weights for positive class
    loss = - (pos_weight * y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
    return tf.reduce_mean(loss)

'''Example usage with different weights'''
pos_weight_1 = 0.5
pos_weight_2 = 0.75

# Calculate weighted binary cross-entropy losses
weighted_loss_1 = weighted_binary_crossentropy(y_true, y_pred, pos_weight_1)
weighted_loss_2 = weighted_binary_crossentropy(y_true, y_pred, pos_weight_2)


## Custom loss function in tensorﬂow to train a model

In [ ]:
'''custom loss function in tensorﬂow/keras to train a model for binary classiﬁcation'''

def custom_loss_with_l2_reg(l2=0.01):
    # Binary cross-entropy loss function
    def custom_loss(y_true, y_pred):
        epsilon = 1e-7  # Small constant to avoid log(0) or log(1)
        
        # Clip values to avoid log(0) or log(1)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
        
        # Calculate binary cross-entropy loss
        binary_loss = - (y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
        binary_loss = tf.reduce_mean(binary_loss)
        
        # Calculate L2 regularization term
        l2_loss = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
        
        # Total loss with L2 regularization
        total_loss = binary_loss + l2 * l2_loss
        return total_loss
    
    return custom_loss


'''apply l2 reguralisation on only convolution and dense layers'''

# Create a Sequential model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss=custom_loss_with_l2_reg(l2=0.01), metrics=['accuracy'])

# Display model summary
model.summary()

#### Assessment:
1. Advantages:
Integrates binary cross-entropy for classification with L2 regularization in a single loss function.
Offers control over model complexity to avoid overfitting.

2. Considerations:
The l2 parameter controls the strength of regularization and needs tuning based on dataset complexity.
Effective for preventing overfitting while optimizing for binary classification.
3. Usage:
Suitable for binary classification tasks requiring simultaneous optimization of cross-entropy and model regularization.
4. Flexibility:
Provides flexibility to adjust the regularization strength for finding the right balance between fitting the data and controlling model complexity.

This combined loss function is useful for binary classification models, offering control over complexity while optimizing for accurate classification. Fine-tuning the regularization strength is crucial for optimal performance based on dataset characteristics and model complexity.